In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
# Making a data frame
Train_data = pd.read_csv('train.txt', delimiter=' ', names = ['Word', 'posTag', 'chunkTag'])

In [5]:
Train_data.head(20)

,Word,posTag,chunkTag
0,Confidence,NN,B-NP
1,in,IN,B-PP
2,the,DT,B-NP
3,pound,NN,I-NP
4,is,VBZ,B-VP
5,widely,RB,I-VP
6,expected,VBN,I-VP
7,to,TO,I-VP
8,take,VB,I-VP
9,another,DT,B-NP


In [6]:
remove = [',', '``', "''", '#', '(', '$', ')', ':']
for item in remove:
    index_names = Train_data[Train_data.posTag == item].index
    Train_data.drop(index_names, inplace=True)

In [7]:
# The unique posTags are:
Tag1 = Train_data.posTag.unique()
print(Tag1.size)
print(Tag1)

# The unique chunkTags are:
Tag2 = Train_data.chunkTag.unique()
print(Tag2.size)
print(Tag2)

36
['NN' 'IN' 'DT' 'VBZ' 'RB' 'VBN' 'TO' 'VB' 'JJ' 'NNS' 'NNP' 'CC' 'POS' '.'
 'VBP' 'VBG' 'PRP$' 'CD' 'VBD' 'EX' 'MD' 'NNPS' 'PRP' 'JJS' 'WP' 'RBR'
 'JJR' 'WDT' 'WRB' 'RBS' 'PDT' 'RP' 'FW' 'WP$' 'SYM' 'UH']
22
['B-NP' 'B-PP' 'I-NP' 'B-VP' 'I-VP' 'B-SBAR' 'B-ADJP' 'O' 'B-ADVP'
 'I-ADVP' 'I-ADJP' 'I-SBAR' 'I-PP' 'B-PRT' 'B-LST' 'B-INTJ' 'I-INTJ'
 'B-CONJP' 'I-CONJP' 'I-PRT' 'B-UCP' 'I-UCP']


## Function to compute emission probabilties for a given word

In [8]:
def EmissionProbablity(word, tag, train_bag = Train_data):
    listOfTags = train_bag[train_bag.posTag == tag]
    tagListCount = len(listOfTags)    
    wordInTag = listOfTags[listOfTags.Word == word]
    wordCountGivenTag = len(wordInTag)
    
    probability = wordCountGivenTag/tagListCount
    return (probability)

## Function to compute transition probabilties for a given tag and previous tag

In [9]:
POS_Tags = Train_data.posTag.unique()
transition_matrix = np.zeros(shape=(len(POS_Tags),len(POS_Tags)))
transition_matrix.shape

(36, 36)

In [10]:
def TransitionProbability(b, a, train_bag = Train_data):
    tags = list(train_bag.posTag)
    a_tags_count = tags.count(a)
    if a_tags_count == 0:
        return 0
    
    b_given_a_list = [i for i in range(len(tags)-1) if tags[i] == a and tags[i+1] == b]
    b_given_a_count = len(b_given_a_list)
    
    return(b_given_a_count/a_tags_count)

Transition Matrix 

In [11]:
for i, tag1 in enumerate(list(POS_Tags)):
    for j, tag2 in enumerate(list(POS_Tags)):
        transition_matrix[i, j] = TransitionProbability(tag2, tag1)

In [12]:
# matrix to DataFrame
tags_df = pd.DataFrame(transition_matrix, columns = list(POS_Tags), index=list(POS_Tags))
tags_df

,NN,IN,DT,VBZ,RB,VBN,TO,VB,JJ,NNS,...,JJR,WDT,WRB,RBS,PDT,RP,FW,WP$,SYM,UH
NN,0.124092,0.264902,0.023817,0.046373,0.023783,0.016221,0.043520,0.002455,0.015524,0.089064,...,0.001758,0.013102,0.002620,0.000365,0.000033,0.000100,0.000166,0.000398,0.000100,0.000000
IN,0.113029,0.030882,0.328062,0.001625,0.015727,0.006018,0.003470,0.000483,0.085486,0.062160,...,0.005403,0.003251,0.001757,0.001889,0.001230,0.000000,0.000264,0.000000,0.000000,0.000000
DT,0.486665,0.008672,0.001854,0.009217,0.011235,0.015817,0.000327,0.000764,0.201309,0.073521,...,0.006545,0.000218,0.000000,0.003054,0.000000,0.000000,0.000218,0.000000,0.000000,0.000218
VBZ,0.040448,0.100043,0.175559,0.003873,0.136403,0.145224,0.045611,0.003442,0.079819,0.018718,...,0.008176,0.000861,0.008391,0.001721,0.000215,0.001506,0.000000,0.000000,0.000000,0.000000
RB,0.012714,0.139549,0.070985,0.044347,0.063115,0.087180,0.026184,0.110338,0.107764,0.009535,...,0.017557,0.000908,0.004389,0.000605,0.000454,0.000000,0.000151,0.000000,0.000151,0.000000
VBN,0.094058,0.368885,0.067814,0.004199,0.045140,0.032543,0.105606,0.001470,0.048079,0.045979,...,0.005669,0.000420,0.003149,0.000210,0.000420,0.000840,0.000000,0.000000,0.000000,0.000000
TO,0.022633,0.004330,0.110411,0.000394,0.009841,0.000984,0.000197,0.584334,0.028538,0.025192,...,0.003543,0.000590,0.000590,0.000394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VB,0.066811,0.144092,0.214559,0.004986,0.047033,0.085591,0.047366,0.007146,0.084261,0.059000,...,0.013960,0.001496,0.005484,0.000831,0.000831,0.004487,0.000000,0.000000,0.000000,0.000000
JJ,0.461139,0.057929,0.008942,0.003133,0.007184,0.003592,0.030264,0.000764,0.077646,0.227589,...,0.000306,0.000611,0.000764,0.000153,0.000076,0.000229,0.000076,0.000076,0.000000,0.000000
NNS,0.026067,0.245392,0.033923,0.014685,0.042661,0.026140,0.042074,0.002350,0.025846,0.020706,...,0.001689,0.017035,0.002643,0.000661,0.000073,0.000000,0.000073,0.000661,0.000000,0.000000


## Viterbi Algorithm on Train data

In [13]:
# Occurance of Tags in the Training Data
dataLength = len(Train_data.posTag)
tagProbability = {}
tags = Train_data.posTag.unique()
for tag in tags:
    tagCount = len(Train_data[Train_data.posTag == tag])
    tagProbability[tag] = tagCount/dataLength
print(tagProbability)

{'NN': 0.1549615770130304, 'IN': 0.11701148834459893, 'DT': 0.09424554730268062, 'VBZ': 0.02389164460664628, 'RB': 0.03396129430208949, 'VBN': 0.024482767483101596, 'TO': 0.026117350741473696, 'VB': 0.03092857693592742, 'JJ': 0.06725950294276388, 'NNS': 0.07000436916908684, 'NNP': 0.10220771543858748, 'CC': 0.027613148628851936, 'POS': 0.009093011899560513, '.': 0.045372535917139994, 'VBP': 0.014742090518903081, 'VBG': 0.016818730884885245, 'PRP$': 0.009668714179238737, 'CD': 0.0427407540671824, 'VBD': 0.03467064175383587, 'EX': 0.0010588809786938754, 'MD': 0.011138811071988487, 'NNPS': 0.0021588835487933384, 'PRP': 0.01963555989616798, 'JJS': 0.0019224343982112107, 'WP': 0.0027191652316944665, 'RBR': 0.0016500038551491942, 'JJR': 0.004384589683620756, 'WDT': 0.004908889974041995, 'WRB': 0.002457015086483847, 'RBS': 0.0009817779948083992, 'PDT': 0.00028271094091341335, 'RP': 0.0004266365108329692, 'FW': 0.00019532755917654014, 'WP$': 0.00017990696239944485, 'SYM': 3.084119355419055e-05

In [14]:
def Viterbi(words, train_bag = Train_data):
    state = [] #Tags
    
    T = train_bag.posTag.unique() # Unique Tags
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        p_transition = [] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = EmissionProbablity(words[key], tag)
            state_probability = emission_p * transition_p
            unknown_word_probability = tagProbability[tag] * transition_p
            p.append(state_probability)
            p_transition.append(unknown_word_probability)
            #faskdjhlskadhj
        pmax = max(p)
        # getting state for which probability is maximum
        if pmax == 0:
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
        else:
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Test Data

Testing data is provided from same https://www.clips.uantwerpen.be/conll2000/chunking/

In [15]:
Test_data = pd.read_csv('test.txt', delimiter=' ', names = ['Word', 'posTag', 'chunkTag'])
Test_data.head(20)

,Word,posTag,chunkTag
0,Rockwell,NNP,B-NP
1,International,NNP,I-NP
2,Corp.,NNP,I-NP
3,'s,POS,B-NP
4,Tulsa,NNP,I-NP
5,unit,NN,I-NP
6,said,VBD,B-VP
7,it,PRP,B-NP
8,signed,VBD,B-VP
9,a,DT,B-NP


### Pre-processing

As we can see that there are lot of special characters like ,''#()$. These need to be removed.

In [16]:
remove = [',', '``', "''", '#', '(', '$', ')', ':']
for item in remove:
    index_names = Test_data[Test_data.posTag == item].index
    Test_data.drop(index_names, inplace=True)

### Testing

In [17]:
test_tagged_words = np.array(Test_data.Word)

In [18]:
start_time = time.time()
tagged_seq = Viterbi(test_tagged_words[:500])
end_time = time.time()
difference_time = end_time-start_time

print("Time taken in seconds: ", difference_time)

Time taken in seconds:  262.4363262653351


In [19]:
# accuracy
check = [i for i, j in zip(tagged_seq, np.array(Test_data.posTag.iloc[:500])) if i[1] == j] 

accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  92.80000000000001
